# ওপেনবিএনবি এমসিপি সার্ভার ইন্টিগ্রেশনের সাথে সেমান্টিক কার্নেল

এই নোটবুকটি দেখায় কীভাবে সেমান্টিক কার্নেল ব্যবহার করে প্রকৃত ওপেনবিএনবি এমসিপি সার্ভারের সাথে এমসিপি স্টুডিও প্লাগইন ব্যবহার করে আসল এয়ারবিএনবি থাকার জায়গা খুঁজে পাওয়া যায়। এলএলএম অ্যাক্সেসের জন্য এটি আজুর এআই ফাউন্ড্রি ব্যবহার করে। আপনার পরিবেশ ভেরিয়েবল সেটআপ করতে, আপনি [সেটআপ লেসন](/00-course-setup/README.md) অনুসরণ করতে পারেন।


## প্রয়োজনীয় প্যাকেজগুলি আমদানি করুন


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP প্লাগইন সংযোগ তৈরি করা

আমরা MCPStdioPlugin ব্যবহার করে [OpenBnB MCP সার্ভার](https://github.com/openbnb-org/mcp-server-airbnb)-এর সাথে সংযোগ করব। এই সার্ভারটি @openbnb/mcp-server-airbnb প্যাকেজের মাধ্যমে Airbnb অনুসন্ধানের কার্যকারিতা প্রদান করে।


## ক্লায়েন্ট তৈরি করা

এই উদাহরণে, আমরা আমাদের LLM অ্যাক্সেসের জন্য Azure AI Foundry ব্যবহার করব। নিশ্চিত করুন যে আপনার পরিবেশ ভেরিয়েবলগুলি সঠিকভাবে সেটআপ করা আছে।


## পরিবেশ কনফিগারেশন

Azure OpenAI সেটিংস কনফিগার করুন। নিশ্চিত করুন যে আপনার নিম্নলিখিত পরিবেশ ভেরিয়েবলগুলো সেট করা আছে:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ইন্টিগ্রেশন বোঝা

এই নোটবুকটি **প্রকৃত OpenBnB MCP সার্ভারের** সাথে সংযুক্ত হয়, যা আসল Airbnb অনুসন্ধানের কার্যকারিতা প্রদান করে।

### এটি কীভাবে কাজ করে:

1. **MCPStdioPlugin**: MCP সার্ভারের সাথে স্ট্যান্ডার্ড ইনপুট/আউটপুট যোগাযোগ ব্যবহার করে
2. **প্রকৃত NPM প্যাকেজ**: `@openbnb/mcp-server-airbnb` প্যাকেজটি npx এর মাধ্যমে ডাউনলোড এবং চালায়
3. **লাইভ ডেটা**: Airbnb এর API থেকে প্রকৃত সম্পত্তির তথ্য প্রদান করে
4. **ফাংশন আবিষ্কার**: এজেন্টটি MCP সার্ভার থেকে উপলব্ধ ফাংশনগুলি স্বয়ংক্রিয়ভাবে আবিষ্কার করে

### উপলব্ধ ফাংশনসমূহ:

OpenBnB MCP সার্ভার সাধারণত নিম্নলিখিত ফাংশনগুলি প্রদান করে:
- **search_listings** - অবস্থান এবং মানদণ্ড অনুযায়ী Airbnb সম্পত্তি অনুসন্ধান করুন
- **get_listing_details** - নির্দিষ্ট সম্পত্তির বিস্তারিত তথ্য পান
- **check_availability** - নির্দিষ্ট তারিখের জন্য প্রাপ্যতা পরীক্ষা করুন
- **get_reviews** - সম্পত্তির জন্য রিভিউ সংগ্রহ করুন
- **get_host_info** - সম্পত্তির হোস্ট সম্পর্কে তথ্য পান

### পূর্বশর্ত:

- আপনার সিস্টেমে **Node.js** ইনস্টল করা থাকতে হবে
- MCP সার্ভার প্যাকেজ ডাউনলোড করার জন্য **ইন্টারনেট সংযোগ** প্রয়োজন
- **NPX** উপলব্ধ থাকতে হবে (Node.js এর সাথে আসে)

### সংযোগ পরীক্ষা:

আপনি MCP সার্ভারটি ম্যানুয়ালি পরীক্ষা করতে পারেন নিম্নলিখিত কমান্ড চালিয়ে:
```bash
npx -y @openbnb/mcp-server-airbnb
```

এটি OpenBnB MCP সার্ভারটি ডাউনলোড এবং চালু করবে, যা পরে Semantic Kernel প্রকৃত Airbnb ডেটার জন্য সংযুক্ত হবে।


## OpenBnB MCP সার্ভারের সাথে এজেন্ট চালানো

এখন আমরা সেই AI এজেন্টটি চালাবো, যা OpenBnB MCP সার্ভারের সাথে সংযুক্ত হয়ে স্টকহোমে ২ জন প্রাপ্তবয়স্ক এবং ১ জন শিশুর জন্য আসল Airbnb থাকার জায়গা খুঁজে বের করবে। অনুসন্ধানের মানদণ্ড পরিবর্তন করতে চাইলে `user_inputs` তালিকাটি পরিবর্তন করতে পারেন।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# সারসংক্ষেপ
অভিনন্দন! আপনি সফলভাবে একটি AI এজেন্ট তৈরি করেছেন যা Model Context Protocol (MCP) ব্যবহার করে বাস্তব জগতের আবাসন অনুসন্ধানের সাথে সংযুক্ত হয়:

## ব্যবহৃত প্রযুক্তি:
- Semantic Kernel - Azure OpenAI ব্যবহার করে বুদ্ধিমান এজেন্ট তৈরি করার জন্য
- Azure AI Foundry - LLM ক্ষমতা এবং চ্যাট সম্পন্ন করার জন্য
- MCP (Model Context Protocol) - মানসম্মত টুল ইন্টিগ্রেশনের জন্য
- OpenBnB MCP Server - বাস্তব Airbnb অনুসন্ধানের কার্যকারিতার জন্য
- Node.js/NPX - বাইরের MCP সার্ভার চালানোর জন্য

## আপনি যা শিখেছেন:
- MCP ইন্টিগ্রেশন: Semantic Kernel এজেন্টকে বাইরের MCP সার্ভারের সাথে সংযুক্ত করা
- রিয়েল-টাইম ডেটা অ্যাক্সেস: লাইভ API ব্যবহার করে বাস্তব Airbnb সম্পত্তি অনুসন্ধান
- প্রোটোকল যোগাযোগ: এজেন্ট এবং MCP সার্ভারের মধ্যে stdio যোগাযোগ ব্যবহার করা
- ফাংশন আবিষ্কার: MCP সার্ভার থেকে উপলব্ধ ফাংশন স্বয়ংক্রিয়ভাবে আবিষ্কার করা
- স্ট্রিমিং রেসপন্স: রিয়েল-টাইমে ফাংশন কল ক্যাপচার এবং লগ করা
- HTML রেন্ডারিং: স্টাইল করা টেবিল এবং ইন্টারঅ্যাকটিভ ডিসপ্লে দিয়ে এজেন্টের রেসপন্স ফরম্যাট করা

## পরবর্তী পদক্ষেপ:
- অতিরিক্ত MCP সার্ভার (আবহাওয়া, ফ্লাইট, রেস্টুরেন্ট) ইন্টিগ্রেট করা
- MCP এবং A2A প্রোটোকল একত্রিত করে একটি মাল্টি-এজেন্ট সিস্টেম তৈরি করা
- আপনার নিজস্ব ডেটা সোর্সের জন্য কাস্টম MCP সার্ভার তৈরি করা
- সেশন জুড়ে স্থায়ী কথোপকথনের স্মৃতি বাস্তবায়ন করা
- MCP সার্ভার অর্কেস্ট্রেশনের সাথে এজেন্টকে Azure Functions-এ ডিপ্লয় করা
- ব্যবহারকারীর প্রমাণীকরণ এবং বুকিং সক্ষমতা যোগ করা



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
